In [1]:
from pyspark.sql import SparkSession
from operator import add

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.166:7077") \
        .appName("sentiment_analysis_group19")\
        .config("spark.executor.cores",4)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/17 12:41:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/17 12:41:17 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [ ]:
lines.take(2)

In [2]:
lines = spark_context.textFile("hdfs://192.168.2.166:9000/input/input/RC_2005-12.bz2")

In [20]:
# pip install vaderSentiment
# pip install nltk

In [14]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...


True

In [22]:
def analyze_sentiment(text):
    """Determines the sentiment of a given text. 
    Returns -1 if text sentiment is neither positive nor negative.
    Returns 1 if text sentiment is mainly positive
    Returns 0 if text sentiment is mainly negative. """
    sia = SentimentIntensityAnalyzer()
    scores = sia.polarity_scores(text)
    pos = scores['pos']
    neg = scores['neg']
    if pos == neg:
        return -1
    elif pos > neg:
        return 1
    else:
        return 0

In [26]:
analyze_sentiment('I am sitting here')

0

In [ ]:
lines.take(2)

In [ ]:
spark_context.stop()